<div style="background-color:#000;"><img src="pqn.png"></img></div>

These libraries help us fetch stock data, analyze options, and visualize results

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from matplotlib import pyplot as plt

### Fetch option data for PLTR

We'll start by getting the options chain for Palantir Technologies (PLTR) with an expiration date of July 18, 2025.

In [ ]:
ticker = "PLTR"
stock = yf.Ticker(ticker)
options_chain = stock.option_chain("2025-07-18")

In [ ]:
calls = options_chain.calls
puts = options_chain.puts

We use the yfinance library to fetch option data for PLTR. We create a Ticker object for PLTR and request its option chain for the specified date. The data is split into two DataFrames: one for call options and another for put options.

### Clean and process option data

Now we'll define functions to clean our data and build an implied volatility grid.

In [ ]:
def clean(df):
    cols = ["strike", "impliedVolatility"]
    return (
        df[cols]
        .dropna(subset=["impliedVolatility"])
        .drop_duplicates("strike")
        .astype({"strike": float, "impliedVolatility": float})
        .sort_values("strike")
        .set_index("strike")
    )

In [ ]:
def build_iv_grid(df):
    df_iv = clean(df)
    strike_grid = np.arange(df_iv.index.min(), df_iv.index.max() + 1, 1.0)
    return (
        df_iv.reindex(strike_grid)
        .interpolate(method="linear")
        .rename_axis("strike")
        .reset_index()
    )

We define two functions: clean and build_iv_grid. The clean function filters and formats our data, keeping only the strike price and implied volatility columns. The build_iv_grid function creates a uniform grid of strike prices and interpolates the implied volatility values. This helps us create a smooth volatility curve.

### Plot volatility skew for calls and puts

Finally, we'll create and plot the volatility skew for both call and put options.

In [ ]:
call_skew = build_iv_grid(calls)

In [ ]:
plt.plot(call_skew.strike, call_skew.impliedVolatility)

In [ ]:
put_skew = build_iv_grid(puts)

In [ ]:
plt.plot(put_skew.strike, put_skew.impliedVolatility)

We apply our build_iv_grid function to both the call and put options data. This gives us a smooth curve of implied volatilities across different strike prices. We then plot these curves using matplotlib. The resulting graph shows the volatility skew for both call and put options, allowing us to visualize how implied volatility changes with the strike price.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.